# Impacto del COVID-19 en el empleo mexicano

In [543]:
import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls
username = 'nestorivanmo'
api_key = 'jUV2OlKD9wX3ZaigY54r'
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Lectura de datos

In [2]:
empleo_df = pd.read_csv('empleo/empleo.csv')
covid_df = pd.read_csv('covid/covid.csv')

In [3]:
print(f"Registros empleo: {empleo_df.shape[0]}\nColumnas: {empleo_df.columns}\n")
print(f"Registros covid: {covid_df.shape[0]}\nColumnas: {covid_df.columns}")

Registros empleo: 196
Columnas: Index(['ANIO', 'MES', 'MES_NUM', 'POB_15_MAS', 'PEA', 'PEA_OCUP', 'PEA_DESOC',
       'EMPLEADORES', '1_SALARIO', '2_SALARIOS', '3_SALARIOS', '5_SALARIOS',
       'MAS_DE_5_SALARIOS', 'SIN_INGRESOS', 'NO_ESPECIFICO_INGRESOS',
       '15_HORAS', '34_HORAS', '48_HORAS', 'MAS_DE_48_HORAS',
       'NO_ESPECIFICO_HORAS', 'TASA_INFORMALIDAD'],
      dtype='object')

Registros covid: 7206669
Columnas: Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'INDIGENA',
       'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'TOMA_MUESTRA_LAB', 'RESULTADO_LAB',
       'TOMA_MUESTRA_ANTIGENO', 'RESULTADO_ANTIGENO', 'C

In [4]:
empleo_df.head()

,ANIO,MES,MES_NUM,POB_15_MAS,PEA,PEA_OCUP,PEA_DESOC,EMPLEADORES,1_SALARIO,2_SALARIOS,...,5_SALARIOS,MAS_DE_5_SALARIOS,SIN_INGRESOS,NO_ESPECIFICO_INGRESOS,15_HORAS,34_HORAS,48_HORAS,MAS_DE_48_HORAS,NO_ESPECIFICO_HORAS,TASA_INFORMALIDAD
0,2005,Enero,1,72721263,42836764,41143856,1692908,2027625,6042705,10138484,...,7132132,4113729,3883275,2187789,2374753,6879338,18885620,11191659,201739,59.137731
1,2005,Febrero,2,72978361,42976836,41355190,1621646,1998350,6068083,10033887,...,7221086,4013786,3614719,2300861,2358087,6625227,19127921,12107405,221132,59.498617
2,2005,Marzo,3,73078716,43198606,41631760,1566846,1804688,6264403,9691136,...,7348548,4209063,4000275,2189573,2747731,8677006,17169588,11067194,186721,59.805053
3,2005,Abril,4,72863898,42399530,40973459,1426071,2065098,5973883,9350592,...,7417377,4356633,3658619,2321402,2388495,6841221,18088761,12273084,225495,59.160939
4,2005,Mayo,5,73128180,43188612,41761793,1426819,1912612,6407787,9818517,...,7474289,4068266,3845147,2207874,2682167,7224529,18572117,12223868,201015,60.148363


In [5]:
covid_df.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2021-06-11,z482b8,1,12,9,2,9,9,12,1,...,2,2,97,2,97,1,99,México,97,97
1,2021-06-11,z49a69,1,12,23,1,23,23,4,2,...,1,2,97,2,97,2,99,México,97,1
2,2021-06-11,z23d9d,1,12,22,2,24,22,9,1,...,2,2,97,2,97,6,99,México,97,97
3,2021-06-11,z24953,1,12,9,1,9,9,10,1,...,1,1,2,2,97,7,99,México,97,97
4,2021-06-11,zz8e77,1,12,9,2,9,9,2,1,...,2,2,97,2,97,6,99,México,97,97


## Visualización del COVID-19
1. Número de casos
2. Número de defunciones

In [6]:
covid_sample_df = covid_df.sample(frac=0.1, random_state=1)

In [7]:
def get_deaths_df(df):
    deaths_df = df[df["FECHA_DEF"] != "9999-99-99"][["FECHA_DEF", "SEXO"]]
    deaths = deaths_df.value_counts()
    deaths_df = deaths.reset_index().rename(columns={0: "NUM_DEF"})
    deaths_df['FECHA_DEF'] = pd.to_datetime(deaths_df["FECHA_DEF"])
    deaths_df = deaths_df.sort_values(by="FECHA_DEF")
    return deaths_df
deaths_df = get_deaths_df(covid_df)

In [8]:
total_deaths = deaths_df['NUM_DEF'].agg('sum')
total_men_deaths = deaths_df[deaths_df['SEXO'] == 2]['NUM_DEF'].agg('sum')
total_women_deaths = deaths_df[deaths_df['SEXO'] == 1]['NUM_DEF'].agg('sum')
print(total_deaths, total_men_deaths, total_women_deaths, total_men_deaths + total_women_deaths)

296929 184154 112775 296929


In [9]:
total_deaths_formatted = '{:,}'.format(total_deaths)
total_men_deaths_formatted = '{:,}'.format(total_men_deaths)
total_women_deaths_formatted = '{:,}'.format(total_women_deaths)

In [546]:
fig = go.Figure()

template = 'plotly_white'
women_line_color = '#48cae4'
men_line_color = '#ef233c'
title_color = "#495057"
subtitle_color = "#adb5bd"

fig.add_trace(go.Scatter(
    x=deaths_df["FECHA_DEF"][deaths_df["SEXO"]==1],
    y=deaths_df["NUM_DEF"][deaths_df["SEXO"]==1],
    name="Mujeres", showlegend=False,
    line=dict(
        color=women_line_color,
        width=2.5
    )
))
fig.add_trace(go.Scatter(
    x=deaths_df["FECHA_DEF"][deaths_df["SEXO"]==2],
    y=deaths_df["NUM_DEF"][deaths_df["SEXO"]==2],
    name="Hombres", showlegend=False,
    line=dict(
        color=men_line_color,
        width=2.5
    )
))
fig.update_layout(
    title=dict(
        text=f"Muertes diarias por COVID-19 en México<br>"
    ),template=template
)
fig.add_annotation(
    text="<b>Hombres<b>",
    font=dict(
        size=15,
        color=men_line_color
    ),
    xref='paper', x=0.54, xanchor='auto',
    yref='paper', y=0.6, yanchor='auto',
    showarrow=False
)
fig.add_annotation(
    text=f"{total_men_deaths_formatted} muertes",
    font=dict(
        size=11,
        color=men_line_color
    ),
    xref='paper', x=0.533, xanchor='auto',
    yref='paper', y=0.548, yanchor='auto',
    showarrow=False
)
fig.add_annotation(
    text="<b>Mujeres<b>",
    font=dict(
        size=15,
        color=women_line_color
    ),
    xref='paper', x=0.32, xanchor='auto',
    yref='paper', y=0.16, yanchor='auto',
    showarrow=False
)
fig.add_annotation(
    text=f"{total_women_deaths_formatted} muertes",
    font=dict(
        size=11,
        color=women_line_color
    ),
    xref='paper', x=0.31, xanchor='auto',
    yref='paper', y=0.1, yanchor='auto',
    showarrow=False
)
fig.add_annotation(
    text=f"<b>En total {total_deaths_formatted} muertes desde <br>Febrero del 2020</b>",
    align='left',
    font=dict(
        size=13,
        color=title_color
    ),
    xref='paper', x=0.03, xanchor='auto',
    yref='paper', y=1, yanchor='auto',
    showarrow=False
)

fig.add_annotation(
    text=f"<b>Primera ola<br>de COVID-19</b>",
    align='center',
    font=dict(
        size=12,
        color=subtitle_color
    ),
    xref='paper', x=0.33, xanchor='auto',
    yref='paper', y=0.65, yanchor='auto',
    arrowcolor=subtitle_color
)

fig.add_annotation(
    text=f"<b>Segunda ola<br>de COVID-19</b>",
    align='center',
    font=dict(
        size=12,
        color=subtitle_color
    ),
    xref='paper', x=0.72, xanchor='auto',
    yref='paper', y=0.95, yanchor='auto',
    arrowcolor=subtitle_color
)
fig.add_annotation(
    text=f"Fuente: Datos abiertos México",
    align='left',
    font=dict(
        size=10,
        color='#777'
    ),
    xref='paper', x=0.99, xanchor='auto',
    yref='paper', y=0.05, yanchor='auto',
    showarrow=False
)
fig.show()

In [547]:
tls.get_embed(py.plot(fig, filename='covid_19_deaths'))

'<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plotly.com/~nestorivanmo/1.embed" height="525" width="100%"></iframe>'

In [11]:
def get_covid_cases_df(df):
    temp_df = df[df["CLASIFICACION_FINAL"].isin([1, 3])][["FECHA_INGRESO", "SEXO"]]
    cases = temp_df.value_counts()
    temp_df = cases.reset_index().rename(columns={0: "NUM_CASES"})
    temp_df["FECHA_INGRESO"] = pd.to_datetime(temp_df["FECHA_INGRESO"])
    temp_df = temp_df.sort_values(by="FECHA_INGRESO")
    return temp_df

In [12]:
cases_df = get_covid_cases_df(covid_df)

In [13]:
total_cases = cases_df["NUM_CASES"].agg('sum')
total_men_cases = cases_df[cases_df['SEXO'] == 2]['NUM_CASES'].agg('sum')
total_women_cases = cases_df[cases_df['SEXO'] == 1]['NUM_CASES'].agg('sum')
print(total_cases, total_men_cases, total_women_cases, total_men_cases + total_women_cases)

2436052 1217521 1218531 2436052


In [14]:
total_cases_formatted = '{:,}'.format(total_cases)
total_men_cases_formatted = '{:,}'.format(total_men_cases)
total_women_cases_formatted = '{:,}'.format(total_women_cases)

In [562]:
template = 'plotly_white'
women_line_color = '#48cae4'
men_line_color = '#ef233c'
title_color = "#343a40"

fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True,
    vertical_spacing=0.1,
    row_heights=[50,50]
)
fig.append_trace(go.Scatter(
    x=cases_df[cases_df["SEXO"]==1]["FECHA_INGRESO"],
    y=cases_df[cases_df["SEXO"]==1]["NUM_CASES"],
    showlegend=False,
    line=dict(
        color=women_line_color,
        width=1,
        shape='vh'
    ),
    stackgroup='one', name='Mujeres'
), row=1, col=1)

fig.append_trace(go.Scatter(
    x=cases_df[cases_df["SEXO"]==2]["FECHA_INGRESO"],
    y=cases_df[cases_df["SEXO"]==2]["NUM_CASES"],
    showlegend=False,
    line=dict(
        color=men_line_color,
        width=1,
        shape='hv'
    ),
    stackgroup='one', name="Hombres"
), row=2, col=1)

fig.update_yaxes(
    showgrid=False,
    row=1, col=1
)

fig.update_yaxes(
    showgrid=False,
    row=2, col=1
)

fig.update_layout(
    title=dict(
        text=f"Casos diarios por COVID-19 en México<br>"
    ),template=template
)

fig.add_annotation(
    text="<b>Hombres<b>",
    font=dict(
        size=15,
        color=men_line_color
    ),
    xref='paper', x=0.08, xanchor='auto',
    yref='paper', y=0.165, yanchor='auto',
    showarrow=False
)
fig.add_annotation(
    text=f"{total_men_cases_formatted} casos",
    font=dict(
        size=11,
        color=men_line_color
    ),
    xref='paper', x=0.075, xanchor='auto',
    yref='paper', y=0.1, yanchor='auto',
    showarrow=False
)
fig.add_annotation(
    text="<b>Mujeres<b>",
    font=dict(
        size=15,
        color=women_line_color
    ),
    xref='paper', x=0.08, xanchor='auto',
    yref='paper', y=0.815, yanchor='auto',
    showarrow=False
)
fig.add_annotation(
    text=f"{total_women_cases_formatted} casos",
    font=dict(
        size=11,
        color=women_line_color
    ),
    xref='paper', x=0.073, xanchor='auto',
    yref='paper', y=0.72, yanchor='auto',
    showarrow=False
)
fig.add_annotation(
    text=f"<b>En total {total_cases_formatted} casos desde <br>Febrero del 2020</b>",
    align='left',
    font=dict(
        size=12,
        color=title_color
    ),
    xref='paper', x=0.5, xanchor='auto',
    yref='paper', y=1, yanchor='auto',
    showarrow=False
)

fig.add_annotation(
    text=f"Fuente: Datos abiertos México",
    align='left',
    font=dict(
        size=10,
        color='#777'
    ),
    xref='paper', x=1, xanchor='auto',
    yref='paper', y=0.5, yanchor='auto',
    showarrow=False
)
fig.show()

In [563]:
tls.get_embed(py.plot(fig, filename='covid_19_cases', auto_open=False))

'<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plotly.com/~nestorivanmo/3.embed" height="525" width="100%"></iframe>'

## Visualización de empleos
1. Tasa de desempleo
2. Tasa de informalidad
3. Tasa de actividad económica
4. Extras

In [16]:
empleo_df.head()

,ANIO,MES,MES_NUM,POB_15_MAS,PEA,PEA_OCUP,PEA_DESOC,EMPLEADORES,1_SALARIO,2_SALARIOS,...,5_SALARIOS,MAS_DE_5_SALARIOS,SIN_INGRESOS,NO_ESPECIFICO_INGRESOS,15_HORAS,34_HORAS,48_HORAS,MAS_DE_48_HORAS,NO_ESPECIFICO_HORAS,TASA_INFORMALIDAD
0,2005,Enero,1,72721263,42836764,41143856,1692908,2027625,6042705,10138484,...,7132132,4113729,3883275,2187789,2374753,6879338,18885620,11191659,201739,59.137731
1,2005,Febrero,2,72978361,42976836,41355190,1621646,1998350,6068083,10033887,...,7221086,4013786,3614719,2300861,2358087,6625227,19127921,12107405,221132,59.498617
2,2005,Marzo,3,73078716,43198606,41631760,1566846,1804688,6264403,9691136,...,7348548,4209063,4000275,2189573,2747731,8677006,17169588,11067194,186721,59.805053
3,2005,Abril,4,72863898,42399530,40973459,1426071,2065098,5973883,9350592,...,7417377,4356633,3658619,2321402,2388495,6841221,18088761,12273084,225495,59.160939
4,2005,Mayo,5,73128180,43188612,41761793,1426819,1912612,6407787,9818517,...,7474289,4068266,3845147,2207874,2682167,7224529,18572117,12223868,201015,60.148363


In [17]:
dates = []
for idx, row in empleo_df.iterrows():
    year = str(row['ANIO'])
    month = str(row['MES_NUM'])
    day = '01'
    if len(month) == 1:
        month = '0' + month
    dates.append(year + '-' + month + '-' + day)
empleo_df['FECHA'] = dates
pd.DataFrame(empleo_df['FECHA'].head())

,FECHA
0,2005-01-01
1,2005-02-01
2,2005-03-01
3,2005-04-01
4,2005-05-01


### Tasa de desempleo

$$\text{Tasa de desempleo} = \frac{\text{PEA}_\text{Desocupada}}{\text{PEA}}$$

$\text{PEA}$ : Población económicamente activa (+15 años)

$\text{PEA}_\text{Desocupada}$: $\text{PEA}$ que no tiene trabajo pero está buscando

### Tasa de actividad

$$\text{Tasa de actividad} = \frac{\text{PEA}}{\text{Población de +15 años}}$$


In [18]:
def get_employment_rates_df(df):
    t = df.loc[:, ["FECHA", "POB_15_MAS", "PEA", "PEA_DESOC", "TASA_INFORMALIDAD"]]
    t["FECHA"] = pd.to_datetime(df["FECHA"])
    t = t.sort_values(by="FECHA")
    t['TASA_DESEMPLEO'] = (t["PEA_DESOC"] / t["PEA"])*100
    t['TASA_ACTIVIDAD'] = (t["PEA"] / t["POB_15_MAS"])*100
    return t[['FECHA', 'TASA_DESEMPLEO', 'TASA_ACTIVIDAD', 'TASA_INFORMALIDAD']]

In [314]:
unemployment_df = get_employment_rates_df(empleo_df)
base_date = '2018-01-01'
unemployment_df = unemployment_df[unemployment_df["FECHA"] >= base_date]
unemployment_df.tail()

,FECHA,TASA_DESEMPLEO,TASA_ACTIVIDAD,TASA_INFORMALIDAD
191,2020-12-01,3.802892,56.406311,55.8375
192,2021-01-01,4.733822,56.097398,55.6374
193,2021-02-01,4.374476,56.719067,55.5085
194,2021-03-01,3.894451,57.173682,56.7976
195,2021-04-01,4.652681,59.053431,55.5917


In [631]:
fig = make_subplots(
    rows=5, cols=2,
    specs=[[{}, {"rowspan": 2}],
           [{}, None],
           [{"rowspan": 2, "colspan": 2}, None],
           [None, None],
           [{'colspan':2}, {}]],
    horizontal_spacing=0.05
)

deep_blue_color = "#0077b6"
rates_line_width = 2.5
red_color_scale = ["#F3DFE1","#F2B9C0","#F1949F","#F16E7E","#F0495D","#EF233C"]
blue_color_scale = ["#D1EEF4","#B6E7F1","#9AE0EE","#7FD8EA","#63D1E7","#48CAE4"]
tick_font_size=9

colorscale=[
    [0, "#48CAE4"],
    [0.1, "#48CAE4"],
    [0.1, "#5BB7D1"],
    [0.2, "#5BB7D1"],
    [0.2, "#6DA5BF"],
    [0.3, "#6DA5BF"],
    [0.3, "#8092AC"],
    [0.4, "#8092AC"],
    [0.4, "#928099"],
    [0.5, "#928099"],
    [0.5, "#A56D87"],
    [0.6, "#A56D87"],
    [0.6, "#B75B74"],
    [0.7, "#B75B74"],
    [0.7, "#CA4861"],
    [0.8, "#CA4861"],
    [0.8, "#DC364F"],
    [0.9, "#DC364F"],
    [0.9, "#EF233C"],
    [1.0, "#EF233C"]
]

fig.add_trace(go.Scatter(
    x=unemployment_df['FECHA'],
    y=unemployment_df['TASA_ACTIVIDAD'],
    showlegend=False, name='Actividad',
    line=dict(
        color=red_color,
        width=rates_line_width
    )
), row=1, col=2)

fig.add_trace(go.Heatmap(
    x=unemployment_df['FECHA'],
    y=[''],
    z=[unemployment_df['TASA_ACTIVIDAD']],
    colorscale=red_color_scale,
    colorbar=dict(
        x=-0.1, xpad=0,
        y=0.82, yanchor='bottom',
        len=0.2, ticksuffix='%', nticks=5,
        ticklabelposition='inside',
        tickfont=dict(
            size=tick_font_size
        )
    )
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=unemployment_df['FECHA'],
    y=unemployment_df['TASA_INFORMALIDAD'],
    showlegend=False, name='Informalidad',
    line=dict(
        color=blue_color,
        width=rates_line_width
    )
), row=1, col=2)

fig.add_trace(go.Heatmap(
    x=unemployment_df['FECHA'],
    y=[''],
    z=[unemployment_df['TASA_INFORMALIDAD']],
    colorscale=blue_color_scale,
    colorbar=dict(
        x=-0.1, xpad=0,
        y=0.61, yanchor='bottom',
        len=0.2, ticksuffix='%', nticks=4,
        ticklabelposition='inside',
        tickfont=dict(
            size=tick_font_size
        )
    )
), row=2, col=1)

deep_yellow = "#faa307"
yellow_color_scale = ["#F0D9AD","#F1D093","#F2C778","#F2BF5E","#F3B643","#F4AD29"]

fig.add_trace(go.Scatter(
    x=unemployment_df['FECHA'],
    y=unemployment_df['TASA_DESEMPLEO'],
    showlegend=False,
    line=dict(
        color=deep_yellow,
        width=1
    ),
    stackgroup='one', name="Desempleo"
), row=3, col=1)

fig.add_trace(go.Heatmap(
    x=unemployment_df['FECHA'],
    y=[''],
    z=[unemployment_df['TASA_DESEMPLEO']],
    colorscale=yellow_color_scale,
    colorbar=dict(
        x=-0.1, xpad=0,
        y=-0.028, yanchor='bottom',
        len=0.2, ticksuffix='%',
        ticklabelposition='inside',
        tickfont=dict(
            size=tick_font_size
        )
    )
), row=5, col=1)

fig.update_layout(
    template=template,
    title=dict(
        text="La primer ola de COVID-19 tuvo un efecto devastador en el empleo mexicano"
    )
)

fig.add_annotation(
    text=f"<b>Tasa de Actividad",
    align='left',
    font=dict(
        size=10,
        color=red_color
    ),
    xref='paper', x=1, xanchor='auto',
    yref='paper', y=1, yanchor='auto',
    showarrow=False
)

fig.add_annotation(
    text=f"<b>Tasa de Informalidad",
    align='left',
    font=dict(
        size=10,
        color=blue_color
    ),
    xref='paper', x=0.65, xanchor='auto',
    yref='paper', y=0.85, yanchor='auto',
    showarrow=False
)

fig.add_annotation(
    text=f"Ambas tasas tuvieron un <b>decremento<br>de aproximadamente 10%</b> durante<br>la primer ola de COVID-19",
    align='center',
    font=dict(
        size=8,
        color=title_color
    ),
    xref='paper', x=0.79, xanchor='auto',
    yref='paper', y=0.77, yanchor='auto',
    showarrow=False
)

fig.add_annotation(
    text=f"<b>Primer ola <br>de COVID-19",
    align='right',
    font=dict(
        size=9,
        color=subtitle_color
    ),
    xref='paper', x=0.97, xanchor='auto',
    yref='paper', y=0.72, yanchor='auto',
    showarrow=False
)

fig.add_annotation(
    text=f"<b>Tasa de Desempleo</b>",
    align='center',
    font=dict(
        size=11,
        color='white'
    ),
    xref='paper', x=0.1, xanchor='auto',
    yref='paper', y=0.37, yanchor='auto',
    showarrow=False
)

fig.add_annotation(
    text=f"<b>Tuvo un incremento del 4.2% al 5.5%<br>durante la primer ola de COVID-19",
    align='center',
    font=dict(
        size=10,
        color='white'
    ),
    xref='paper', x=0.06, xanchor='auto',
    yref='paper', y=0.245, yanchor='auto',
    showarrow=False
)

fig.add_annotation(
    text=f"<b>Primer ola <br>de COVID-19",
    align='center',
    font=dict(
        size=9,
        color='white'
    ),
    xref='paper', x=0.84, xanchor='auto',
    yref='paper', y=0.49, yanchor='auto',
    showarrow=False
)

fig.add_annotation(
    text=f"<b>Actividad económica",
    align='center',
    font=dict(
        size=10,
        color='white'
    ),
    xref='paper', x=0.01, xanchor='auto',
    yref='paper', y=0.99, yanchor='auto',
    showarrow=False
)

fig.add_annotation(
    text=f"<b>Informalidad laboral",
    align='center',
    font=dict(
        size=10,
        color='white'
    ),
    xref='paper', x=0.01, xanchor='auto',
    yref='paper', y=0.78, yanchor='auto',
    showarrow=False
)

fig.add_annotation(
    text=f"<b>Desempleo",
    align='center',
    font=dict(
        size=11,
        color='white'
    ),
    xref='paper', x=0.993, xanchor='auto',
    yref='paper', y=0.01, yanchor='auto',
    showarrow=False
)

fig.add_annotation(
    text=f"Fuente: Encuesta Nacional de Ocupación y Empleo 2005-2021",
    align='left',
    font=dict(
        size=8,
        color='#ffe'
    ),
    xref='paper', x=0, xanchor='auto',
    yref='paper', y=0, yanchor='auto',
    showarrow=False
)

fig.show()

In [632]:
tls.get_embed(py.plot(fig, filename='rates', auto_open=False))

'<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plotly.com/~nestorivanmo/30.embed" height="525" width="100%"></iframe>'

### 4. Extras

In [307]:
empleo_df.columns

Index(['ANIO', 'MES', 'MES_NUM', 'POB_15_MAS', 'PEA', 'PEA_OCUP', 'PEA_DESOC',
       'EMPLEADORES', '1_SALARIO', '2_SALARIOS', '3_SALARIOS', '5_SALARIOS',
       'MAS_DE_5_SALARIOS', 'SIN_INGRESOS', 'NO_ESPECIFICO_INGRESOS',
       '15_HORAS', '34_HORAS', '48_HORAS', 'MAS_DE_48_HORAS',
       'NO_ESPECIFICO_HORAS', 'TASA_INFORMALIDAD', 'FECHA'],
      dtype='object')

In [325]:
empleo_df.head()

,ANIO,MES,MES_NUM,POB_15_MAS,PEA,PEA_OCUP,PEA_DESOC,EMPLEADORES,1_SALARIO,2_SALARIOS,...,MAS_DE_5_SALARIOS,SIN_INGRESOS,NO_ESPECIFICO_INGRESOS,15_HORAS,34_HORAS,48_HORAS,MAS_DE_48_HORAS,NO_ESPECIFICO_HORAS,TASA_INFORMALIDAD,FECHA
0,2005,Enero,1,72721263,42836764,41143856,1692908,2027625,6042705,10138484,...,4113729,3883275,2187789,2374753,6879338,18885620,11191659,201739,59.137731,2005-01-01
1,2005,Febrero,2,72978361,42976836,41355190,1621646,1998350,6068083,10033887,...,4013786,3614719,2300861,2358087,6625227,19127921,12107405,221132,59.498617,2005-02-01
2,2005,Marzo,3,73078716,43198606,41631760,1566846,1804688,6264403,9691136,...,4209063,4000275,2189573,2747731,8677006,17169588,11067194,186721,59.805053,2005-03-01
3,2005,Abril,4,72863898,42399530,40973459,1426071,2065098,5973883,9350592,...,4356633,3658619,2321402,2388495,6841221,18088761,12273084,225495,59.160939,2005-04-01
4,2005,Mayo,5,73128180,43188612,41761793,1426819,1912612,6407787,9818517,...,4068266,3845147,2207874,2682167,7224529,18572117,12223868,201015,60.148363,2005-05-01


In [399]:
empleo_df = pd.read_csv('empleo/empleo.csv')
dates = []
for idx, row in empleo_df.iterrows():
    year = str(row['ANIO'])
    month = str(row['MES_NUM'])
    day = '01'
    if len(month) == 1:
        month = '0' + month
    dates.append(year + '-' + month + '-' + day)
empleo_df['FECHA'] = dates
base_date = '2008-01-01'
empleo_df = empleo_df[empleo_df['FECHA'] > base_date]

In [627]:
fig = make_subplots(
    rows=5, cols=4,
    specs=[
        [{},{'rowspan':5, 'colspan':3}, None, None],
        [{},None,None,None],
        [{},None,None,None],
        [{},None,None,None],
        [{},None,None,None]
    ],
    shared_xaxes=True,
    subplot_titles=(
        "<b>Número de empleadores vs.</b><br>personas con hasta 1 salario mínimo",
        "",
        "<b>Número de empleadores vs.</b><br>Personas con hasta 2 salarios mínimos",
        "<b>Número de empleadores vs.</b><br>Personas con hasta 3 salarios mínimos",
        "<b>Número de empleadores vs.</b><br>Personas con hasta 5 salarios mínimos",
        "<b>Número de empleadores vs.</b><br>Personas con más de 5 salarios mínimos",
    )
)

orange_color = '#f4a261'
deep_blue = '#023e8a'
second_blue = '#457b9d'
gray_1 = "#DEE2E6"
gray_2 = "#94999F"
gray_3 = "#495057"
line_colors = [gray_1, red_color, gray_2, gray_3, blue_color]

for idx, column_name in enumerate(['1_SALARIO', '2_SALARIOS', '3_SALARIOS', '5_SALARIOS', 'MAS_DE_5_SALARIOS']):
    fig.add_trace(go.Scatter(
        x=empleo_df['FECHA'],
        y=empleo_df[column_name],
        name=column_name, showlegend=False,
        line=dict(
            width=2.4,
            shape='spline',
            smoothing=1,
            color=line_colors[idx]
        )
    ), row=1, col=2)

    fig.add_trace(go.Scatter(
        x=empleo_df['EMPLEADORES'],
        y=empleo_df[column_name],
        name=column_name, mode='markers',
        showlegend=False,
        marker=dict(
            size=3,
            opacity=1,
            color=line_colors[idx]
        )
    ), row=idx+1, col=1)


fig.update_layout(
    template=template,
    title=dict(
        text='Incrementando el número de empleadores se pueden incrementar la el número<br>de personas con 1 y 2 salarios mínimos',
        font_size=13
    )
)

fig.update_annotations(font_size=9)
fig.update_xaxes(title_text="<b>Salarios", title_font_size=9, row=5, col=1)
fig.update_xaxes(title_text="", title_font_size=9, row=1, col=1)
fig.update_xaxes(title_text="", title_font_size=9, row=2, col=1)
fig.update_xaxes(title_text="", title_font_size=9, row=3, col=1)
fig.update_xaxes(title_text="", title_font_size=9, row=4, col=1)

fig.add_annotation(
    text=f"<b>Distribución de salarios mínimos</b><br>desde enero del 2008",
    align='left',
    font=dict(
        size=15,
        color=title_color
    ),
    xref='paper', x=0.28, xanchor='auto',
    yref='paper', y=1, yanchor='auto',
    showarrow=False
)

salaries_annotation_size = 8

fig.add_annotation(
    text=f"<b>2 salarios mínimos",
    align='center',
    font=dict(
        size=salaries_annotation_size,
        color=red_color
    ),
    xref='paper', x=1, xanchor='auto',
    yref='paper', y=0.99, yanchor='auto',
    showarrow=False
)

fig.add_annotation(
    text=f"<b>3 salarios mínimos",
    align='center',
    font=dict(
        size=salaries_annotation_size,
        color=gray_2
    ),
    xref='paper', x=0.88, xanchor='auto',
    yref='paper', y=0.58, yanchor='auto',
    showarrow=False
)

fig.add_annotation(
    text=f"<b>Más de 5 salarios mínimos",
    align='center',
    font=dict(
        size=salaries_annotation_size,
        color=blue_color
    ),
    xref='paper', x=0.3, xanchor='auto',
    yref='paper', y=0.13, yanchor='auto',
    showarrow=False
)

fig.add_annotation(
    text=f"<b>5 salarios mínimos",
    align='center',
    font=dict(
        size=salaries_annotation_size,
        color=gray_3
    ),
    xref='paper', x=0.92, xanchor='auto',
    yref='paper', y=0.17, yanchor='auto',
    showarrow=False
)

fig.add_annotation(
    text=f"<b>1 salario mínimo",
    align='center',
    font=dict(
        size=salaries_annotation_size,
        color=gray_1
    ),
    xref='paper', x=0.53, xanchor='auto',
    yref='paper', y=0.25, yanchor='auto',
    showarrow=False
)

fig.add_trace(go.Scatter(
    x=empleo_df['FECHA'],
    y=[5000000]*empleo_df['FECHA'].shape[0],
    line=dict(
        color=deep_yellow,
        width=1,
        dash="dashdot"
    ),showlegend=False
),row=1, col=2)

fig.add_annotation(
    text=f"Fuente: Encuesta Nacional de Ocupación y Empleo 2005-2021",
    align='left',
    font=dict(
        size=8,
        color=subtitle_color
    ),
    xref='paper', x=0.26, xanchor='auto',
    yref='paper', y=0.02, yanchor='auto',
    showarrow=False
)


fig.show()

In [628]:
tls.get_embed(py.plot(fig, filename='salaries', auto_open=False))

'<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plotly.com/~nestorivanmo/7.embed" height="525" width="100%"></iframe>'

In [534]:
print(red_color, blue_color, deep_yellow)

#ef233c #48cae4 #faa307


In [630]:
title_color

'#343a40'

In [21]:
fig.show()

In [20]:
template = 'plotly_white'
blue_color = '#48cae4'
red_color = '#ef233c'
title_color = "#343a40"
subtitle_color = "#adb5bd"

colorscale=["#48CAE4","#72A0BA","#9C7790","#C54D66","#EF233C"]
#colorscale='Plasma'

fig = make_subplots(
    rows=3, cols=1, shared_xaxes=True,
    vertical_spacing=0.08
)

for idx, tasa in enumerate(['TASA_ACTIVIDAD', 'TASA_INFORMALIDAD', 'TASA_DESEMPLEO']):
    fig.add_trace(go.Heatmap(
        z=[
            unemployment_df[tasa]
        ],
        y=[tasa.split('_')[1].capitalize()],
        x=unemployment_df['FECHA'],
        colorscale=colorscale,
        colorbar=dict(
            len=0.1,
            nticks=10,
            ticks='outside',
            ticksuffix='%',
            title=dict(
                text='Porcentaje'
            )
        )
    ), row=idx+1, col=1)

fig.update_layout(
    title=dict(
        text="Principales indicadores del empleo"
    ),
    width=1000,height=400
)
fig.show()